# Acidentes de tráfico de bicicletas

Vamos a utilizar un dataset de accidentes de tráfico de bicicletas que provienen del portal de datos abiertos de Madrid

https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=20f4a87ebb65b510VgnVCM1000001d4a900aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default


Una vez preprocesados los datos tendremos un dataset con los siguientes campos:


- Número de expediente del accidente
- Fecha del accidente
- Distrito donde ha ocurrido el accidente
- Tipo de accidente
- Sexo de la persona que ha tenido el accidente
    

Vamos a intentar resolver las siguientes preguntas:

- Datos del número de expediente 2020S000044
- Accidentes que ocurren en el distrito de Moratalaz ordenados por fecha (Descendente)
- Número de accidentes por tipo de accidente (sin ningún orden establecido)
- Número de accidentes cuyo tipo es Colisión frontal

Preprocesado de datos

Se realizará un preprocesado de datos cuyo objetico es limpiar, simplificar y normalizar los datos para que la prática resulte más sencilla

El formato de salida será un fichero Microsoft Excel que contendrá los datos limpios.



## Carga de datos

In [28]:
import pandas as pd

Leemos los datos ...

In [29]:
df = pd.read_csv("data/AccidentesBicicletas_2020.txt", delimiter=";")

## Exploración y limpieza

Limpiamos los datos ...

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 775 entries, 0 to 774
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   NumExpediente        775 non-null    object 
 1   Fecha                775 non-null    object 
 2   Hora                 775 non-null    object 
 3   Calle                775 non-null    object 
 4   Numero               775 non-null    object 
 5   Distrito             775 non-null    object 
 6   TipoAccidente        775 non-null    object 
 7   EstadoMetereologico  759 non-null    object 
 8   TipoVehiculo         775 non-null    object 
 9   TipoPersona          775 non-null    object 
 10  RangoEdad            775 non-null    object 
 11  Sexo                 763 non-null    object 
 12  Lesividad            684 non-null    float64
dtypes: float64(1), object(12)
memory usage: 78.8+ KB


In [31]:
df = df.assign(Fecha = pd.to_datetime(df.Fecha, format="%d/%m/%Y"),
               Min = pd.to_numeric(df.Hora.str.split(":").str.get(1)),
               Hora = pd.to_numeric(df.Hora.str.split(":").str.get(0))
              )
df = df.filter(["NumExpediente", "Fecha", "Distrito", "TipoAccidente", "Sexo"])
# Ponemos las columnas en minúsculas para no se inserten en Postgres con Dobles Comillas
df = df.rename(columns = {"NumExpediente" : "num_expediente",
                           "Fecha" : "fecha",
                           "Distrito": "distrito",
                           "TipoAccidente" : "tipo_accidente",
                           "Sexo": "sexo"}
              )
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 775 entries, 0 to 774
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   num_expediente  775 non-null    object        
 1   fecha           775 non-null    datetime64[ns]
 2   distrito        775 non-null    object        
 3   tipo_accidente  775 non-null    object        
 4   sexo            763 non-null    object        
dtypes: datetime64[ns](1), object(4)
memory usage: 30.4+ KB


In [32]:
df.sample(10)

,num_expediente,fecha,distrito,tipo_accidente,sexo
365,2020S008713,2020-06-23,CHAMBERÍ,Alcance,Hombre
364,2020S008701,2020-06-23,CHAMBERÍ,Otro,Hombre
478,2020S010311,2020-07-23,VILLAVERDE,Colisión fronto-lateral,Hombre
219,2020S007244,2020-05-25,CENTRO,Caída,Hombre
435,2020S009674,2020-07-10,RETIRO,Caída,Hombre
695,2020S013922,2020-10-05,CHAMARTÍN,Caída,Mujer
201,2020S007104,2020-05-21,HORTALEZA,Colisión fronto-lateral,Hombre
177,2020S006829,2020-05-13,CENTRO,Caída,Hombre
574,2020S011620,2020-08-25,TETUÁN,Alcance,Hombre
763,2020S015311,2020-10-29,RETIRO,Alcance,Hombre


Eliminamos los expedientes duplicados para simplificar el ejemplo ...

In [33]:
ExpedientesDuplicados = pd.DataFrame(df.groupby("num_expediente").size()).reset_index().rename( columns = {0 : 'n'}).query("n > 1").num_expediente
df = df[~df.num_expediente.isin(ExpedientesDuplicados)]
df = df[df.sexo.notnull()]
df = df.reset_index(drop = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   num_expediente  614 non-null    object        
 1   fecha           614 non-null    datetime64[ns]
 2   distrito        614 non-null    object        
 3   tipo_accidente  614 non-null    object        
 4   sexo            614 non-null    object        
dtypes: datetime64[ns](1), object(4)
memory usage: 24.1+ KB


In [34]:
df.sample(10)

,num_expediente,fecha,distrito,tipo_accidente,sexo
583,2020S014570,2020-10-18,HORTALEZA,Alcance,Hombre
233,2020S008055,2020-06-09,USERA,Colisión frontal,Hombre
457,2020S011650,2020-08-26,RETIRO,Colisión fronto-lateral,Mujer
543,2020S013704,2020-10-03,CIUDAD LINEAL,Caída,Hombre
352,2020S009691,2020-07-11,ARGANZUELA,Caída,Mujer
342,2020S010819,2020-07-07,MONCLOA-ARAVACA,Caída,Mujer
55,2020S003108,2020-02-08,HORTALEZA,Caída,Hombre
580,2020S014578,2020-10-18,RETIRO,Choque contra obstáculo fijo,Hombre
71,2020S003961,2020-02-20,RETIRO,Alcance,Hombre
236,2020S008184,2020-06-10,HORTALEZA,Caída,Mujer


## Grabación de los datos limpios

Y finalmente escribimos el resultado a un fichero Excel ,,,

In [35]:
writer = pd.ExcelWriter("data/accidentes.xlsx")
df.to_excel(writer, 'Data', index=False)
writer.save()